## 쇼핑몰 크롤링 DB에 저장

- 데이터베이스 생성

-  'create database beauty_shop'

In [16]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import pymysql

#### 모든 제품을 크롤링해서 DB에 저장
- 저장할 DB부터 테이블 생성해서 형식에 맞게 data 변환 후 DB로 insert
1. DB 연결 코드
2. 필요한 DB 및 table 생성
3. 크롤링
    - 데이터를 table 형식에 맞게 정제
4. insert (데이터 생성)
5. commit() 해서 db에 반영
6. db 닫기

- db 테이블을 읽어와서 df 에 저장

1. DB연결 및 필요 객체 생성

In [20]:
def conn(d_name) :
    import pymysql
    host_name = 'localhost'
    host_port = 3306
    username = 'root'
    password = 'toor'
    database_name = d_name
    db = pymysql.connect(
        host=host_name,     # MySQL Server Address
        port=host_port,          # MySQL Server Port
        user=username,      # MySQL username
        passwd=password,    # password for MySQL username
        db=database_name,   # Database name
        charset='utf8'
    )
    return db

In [22]:
db = conn('beauty_shop')
cursor = db.cursor()

In [24]:
sql = "show databases"
cursor.execute(sql)
result = cursor.fetchall()
result

(('beauty_shop',),
 ('ecommerce',),
 ('employees',),
 ('information_schema',),
 ('mysql',),
 ('performance_schema',),
 ('sakila',),
 ('sqldb',),
 ('sys',),
 ('world',))

In [26]:
sql = "use beauty_shop"
cursor.execute(sql)

0

In [28]:
sql = "select database()"
cursor.execute(sql)
result = cursor.fetchone()
result

('beauty_shop',)

In [30]:
 # table 생성
 sql = '''
     CREATE TABLE product (
         PRODUCT_CODE int AUTO_INCREMENT NOT NULL,
         TITLE VARCHAR(200) NOT NULL,
         ORI_PRICE VARCHAR(100),
         DISCOUNT_PRICE VARCHAR(100),
         link VARCHAR(200),
         PRIMARY KEY(PRODUCT_CODE)
     );
 '''

 cursor.execute(sql)
 db.commit()

In [32]:
sql = "show tables"
cursor.execute(sql)
result = cursor.fetchall()
result

(('product',),)

In [34]:
sql = 'desc product'
cursor.execute(sql)
result = cursor.fetchall()
result

(('PRODUCT_CODE', 'int', 'NO', 'PRI', None, 'auto_increment'),
 ('TITLE', 'varchar(200)', 'NO', '', None, ''),
 ('ORI_PRICE', 'varchar(100)', 'YES', '', None, ''),
 ('DISCOUNT_PRICE', 'varchar(100)', 'YES', '', None, ''),
 ('link', 'varchar(200)', 'YES', '', None, ''))

2. 크롤링 코드 - insert 구문을 추가해서 변경

In [37]:
# 크롤링 문서 요청해서 응답객체 반환(첫번째 크롤링 문서 요청 후 응답)
url="http://jolse.com/category/toners-mists/1019/"
html = urlopen(url)
htmls = html.read()
# print(htmls)
bs_obj = BeautifulSoup(htmls,"html.parser")

In [38]:
# box안에 들어 있는 1개의 상품에서 정보를 추출해서 dict형태로  반환하는 함수
# 데이터 전처리 : 제품명에 ' 제거/ 가격 USD 제거 / 세일가격 없는 경우 처리
def get_product_info(box) :
    strong_tag = box.find("strong",{"class":"name"})
    # 품목 추출
    span =strong_tag.text.split(':')[1]
    
    # 세부페이지링크 추출
    a = strong_tag.find("a")
    sub_link = 'https://jolse.com' + a["href"]
    # 가격 추출 코드
    price_ul = box.find("ul")
    price_span = "'"+price_ul.findAll("span")[1].text+"'"
    sals_price = "'"+box.find('ul').findAll('span')[-1].text+"'"
    
    # 데이터 전처리
    title = span.replace("'","''") # ' 처리
    ord_price = price_span
    dis_price = sals_price
   # ord_price = price_span[1].text.split(' ')[1] # USD 제거
   # dis_price = price_span[-1].text.split(' ')[1]
    # 세일 가격이 없는 경우 
    if dis_price =='' :
        dis_price = '0.0'
    
    
    # 최종 data 추출 후 반환 
    return{"prd_name":title,"price":ord_price,"sale_price":dis_price,"sub_link":sub_link}


In [41]:
def save_data(prd_info) :
#     print(prd_info)
    
    # insert 구문
    sql = "INSERT INTO product (title, ori_price, discount_price,link) values('" \
        + prd_info["prd_name"] \
        +"'," \
        + prd_info['price']\
        +"," \
        + prd_info['sale_price']\
        +",'"\
        + prd_info['sub_link']\
        + "');"
    print(sql)
    cursor.execute(sql)

In [43]:
# 전달된 url 페이지에 접근해서 해당페지의 전체 상품 데이터를 추출 한 후 각 상품마다 get_product_info()함수를 호출해서
# 각 상품에대한 추출 정보를 받아옴 - 들어온 각 상품 정보를 리스트에 저장 한 수 해당 반환
def get_page_products(url) :
    url=url
    html = urlopen(url)
    htmls = html.read()
    # print(htmls)
    bs_obj = BeautifulSoup(htmls,"html.parser")

    ## 한 페이지에 모든 상품이 들어있는 ul 태그 추출
    # ul class:prdList grid4
    ul=bs_obj.find("ul",{"class":"prdList grid5"})
    ## 품목 1개를 담고 있는 div 태그 추출
    ## div class:box
    prd_boxes = ul.findAll("div", {"class":"description"}) #1개 페이지의 전체 상품
    # 반환되는 품목 데이터를 db에 insert  : 함수호출해서 진행
    for box in prd_boxes :
        prd = get_product_info(box)
        print(prd)
        save_data(prd) #사용자 정의 함수(생성해야 함)

### main 코드(프로그램 시작점)

In [64]:
from tqdm.notebook import tqdm # 상태바 표시

#여러 페이지의 화장품 정보를 추출해서 df 에 저장 후 csv에 저장하는 코드
url = "http://jolse.com/category/toners-mists/1019/?page=" #페이지 번호를 제외한 공통 url 문자열

#last= int(bs_obj.find("p",{"class":"last"}).find("a")['href'].split("=")[1])
last=3
for i in tqdm(range(2,last+1)) : # 2페이지부터 수집
# for i in range(1,2) : # 1페이지 insert
    # url 완성 :page번호를 추가 i 변수 값을 활용
    urlfin =url + str(i)
    get_page_products(urlfin)

  0%|          | 0/2 [00:00<?, ?it/s]

{'prd_name': ' ANUA BHA 2% Gentle Exfoliating Toner 150ml', 'price': "'USD 25.00'", 'sale_price': "'(15% OFF)'", 'sub_link': 'https://jolse.com/product/anua-bha-2-gentle-exfoliating-toner-150ml/75476/category/1019/display/1/'}
INSERT INTO product (title, ori_price, discount_price,link) values(' ANUA BHA 2% Gentle Exfoliating Toner 150ml','USD 25.00','(15% OFF)','https://jolse.com/product/anua-bha-2-gentle-exfoliating-toner-150ml/75476/category/1019/display/1/');
{'prd_name': ' ROUND LAB 1025 Dokdo On The Go Kit', 'price': "'USD 23.00'", 'sale_price': "'(15% OFF)'", 'sub_link': 'https://jolse.com/product/round-lab-1025-dokdo-on-the-go-kit/52758/category/1019/display/1/'}
INSERT INTO product (title, ori_price, discount_price,link) values(' ROUND LAB 1025 Dokdo On The Go Kit','USD 23.00','(15% OFF)','https://jolse.com/product/round-lab-1025-dokdo-on-the-go-kit/52758/category/1019/display/1/');
{'prd_name': ' Abib Heartleaf calming toner (miniature) Skin booster 30ml', 'price': "'USD 7.00'

In [66]:
db.commit()

In [68]:
sql = "select * from product"
cursor.execute(sql)
result = cursor.fetchall()
result

((1,
  ' ANUA BHA 2% Gentle Exfoliating Toner 150ml',
  'USD 25.00',
  '(15% OFF)',
  'https://jolse.com/product/anua-bha-2-gentle-exfoliating-toner-150ml/75476/category/1019/display/1/'),
 (2,
  ' ROUND LAB 1025 Dokdo On The Go Kit',
  'USD 23.00',
  '(15% OFF)',
  'https://jolse.com/product/round-lab-1025-dokdo-on-the-go-kit/52758/category/1019/display/1/'),
 (3,
  ' Abib Heartleaf calming toner (miniature) Skin booster 30ml',
  'USD 7.00',
  '(15% OFF)',
  'https://jolse.com/product/abib-heartleaf-calming-toner-miniature-skin-booster-30ml/73817/category/1019/display/1/'),
 (4,
  " A'PIEU Raspberry Vinegar Hair Mist 105ml",
  'USD 16.07',
  '(36% OFF)',
  'https://jolse.com/product/apieu-raspberry-vinegar-hair-mist-105ml/15720/category/1019/display/1/'),
 (5,
  ' numbuzin No.1 Pure-Full Calming Herb Toner 100ml',
  'USD 11.00',
  '(20% OFF)',
  'https://jolse.com/product/numbuzin-no1-pure-full-calming-herb-toner-100ml/65098/category/1019/display/1/'),
 (6,
  ' Anua Heartleaf 77% Clea

In [70]:
db.close()

## db 테이블에 저장된 데이터 df로 가져오기

In [73]:
db = conn('beauty_shop')

In [75]:
sql = "select * from product"
df = pd.read_sql(sql,db)
df

C:\Users\user\AppData\Local\Temp\ipykernel_11340\3017133393.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql,db)


,PRODUCT_CODE,TITLE,ORI_PRICE,DISCOUNT_PRICE,link
0,1,ANUA BHA 2% Gentle Exfoliating Toner 150ml,USD 25.00,(15% OFF),https://jolse.com/product/anua-bha-2-gentle-ex...
1,2,ROUND LAB 1025 Dokdo On The Go Kit,USD 23.00,(15% OFF),https://jolse.com/product/round-lab-1025-dokdo...
2,3,Abib Heartleaf calming toner (miniature) Skin...,USD 7.00,(15% OFF),https://jolse.com/product/abib-heartleaf-calmi...
3,4,A'PIEU Raspberry Vinegar Hair Mist 105ml,USD 16.07,(36% OFF),https://jolse.com/product/apieu-raspberry-vine...
4,5,numbuzin No.1 Pure-Full Calming Herb Toner 100ml,USD 11.00,(20% OFF),https://jolse.com/product/numbuzin-no1-pure-fu...
...,...,...,...,...,...
395,396,ASIS-TOBE Forest Relaxing Toner 200ml,USD 21.00,(15% OFF),https://jolse.com/product/asis-tobe-forest-rel...
396,397,Real Barrier Extreme Essence Toner 200ml,USD 30.00,(50% OFF),https://jolse.com/product/real-barrier-extreme...
397,398,BLITHE Vital Treatment 8 Nourishing Beans 150ml,USD 42.90,(15% OFF),https://jolse.com/product/blithe-vital-treatme...
398,399,HAYEJIN RiceFila™ Oil Toner 150ml,USD 30.00,(15% OFF),https://jolse.com/product/hayejin-ricefila-oil...


In [77]:
sql = "select title,ori_price,discount_price from product"
df = pd.read_sql(sql,db)
df

C:\Users\user\AppData\Local\Temp\ipykernel_11340\2998014182.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql,db)


,title,ori_price,discount_price
0,ANUA BHA 2% Gentle Exfoliating Toner 150ml,USD 25.00,(15% OFF)
1,ROUND LAB 1025 Dokdo On The Go Kit,USD 23.00,(15% OFF)
2,Abib Heartleaf calming toner (miniature) Skin...,USD 7.00,(15% OFF)
3,A'PIEU Raspberry Vinegar Hair Mist 105ml,USD 16.07,(36% OFF)
4,numbuzin No.1 Pure-Full Calming Herb Toner 100ml,USD 11.00,(20% OFF)
...,...,...,...
395,ASIS-TOBE Forest Relaxing Toner 200ml,USD 21.00,(15% OFF)
396,Real Barrier Extreme Essence Toner 200ml,USD 30.00,(50% OFF)
397,BLITHE Vital Treatment 8 Nourishing Beans 150ml,USD 42.90,(15% OFF)
398,HAYEJIN RiceFila™ Oil Toner 150ml,USD 30.00,(15% OFF)
